<a href="https://colab.research.google.com/github/AmritSDutta/colab_ml/blob/main/basic_geospatial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import time
from tqdm import tqdm

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import requests

In [61]:
#Sample data
df = pd.read_csv(r'cities.csv')
print(df.head(5))

   id         name country  latitude  longitude  population
0   1        Bombo     UGA    0.5833    32.5333       75000
1   2  Fort Portal     UGA    0.6710    30.2750       42670
2   3      Potenza     ITA   40.6420    15.7990       69060
3   4   Campobasso     ITA   41.5630    14.6560       50762
4   5        Aosta     ITA   45.7370     7.3150       34062


**Add elevation**

In [63]:
df.rename(columns={'latitude': 'lat', 'longitude': 'lon'}, inplace=True)
elevations = []
for index, row in tqdm(df.iterrows()):
    lat, lon = (row['lat'], row['lon'])
    # changeable URL
    url = f"https://api.opentopodata.org/v1/aster30m?locations={lat},{lon}"
    r = requests.get(url)
    data = r.json()
    if data.get('results') and len(data['results']) > 0:
      elevation = data['results'][0]['elevation']
    else:
      elevation = None
    elevations.append(elevation)
    time.sleep(0.8)
    if index % 100 == 0:
        print(f"Processed {index} rows")
        time.sleep(2.5)


df['elevation'] = elevations
print(df.head(5))
df.to_csv('city_elevation.csv', index=True)

0it [00:00, ?it/s]

Processed 0 rows


100it [01:54,  1.08s/it]

Processed 100 rows


200it [03:45,  1.13s/it]

...

Processed 1100 rows


1200it [22:28,  1.09s/it]

Processed 1200 rows


1249it [23:24,  1.12s/it]

   id         name country      lat      lon  population  elevation
0   1        Bombo     UGA   0.5833  32.5333       75000     1191.0
1   2  Fort Portal     UGA   0.6710  30.2750       42670     1517.0
2   3      Potenza     ITA  40.6420  15.7990       69060      714.0
3   4   Campobasso     ITA  41.5630  14.6560       50762      768.0
4   5        Aosta     ITA  45.7370   7.3150       34062      575.0


In [64]:
# Create GeoDataFrame
geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')
gdf.head()

,id,name,country,lat,lon,population,elevation,geometry
0,1,Bombo,UGA,0.5833,32.5333,75000,1191.0,POINT (32.5333 0.5833)
1,2,Fort Portal,UGA,0.6710,30.2750,42670,1517.0,POINT (30.275 0.671)
2,3,Potenza,ITA,40.6420,15.7990,69060,714.0,POINT (15.799 40.642)
3,4,Campobasso,ITA,41.5630,14.6560,50762,768.0,POINT (14.656 41.563)
4,5,Aosta,ITA,45.7370,7.3150,34062,575.0,POINT (7.315 45.737)


**Distance to coastline (approx, using Natural Earth coastline)**

In [69]:
coastline = gpd.read_file(r'ne_110m_coastline.shp')
gdf['proximity_to_coast_km'] = gdf.geometry.apply(
    lambda city: coastline.distance(city).min() * 111)  # Rough degree to km

**Distance to fresh water (approx, using Natural Earth coastline)**

In [70]:
# Feature 3: Distance to freshwater (approx, using Natural Earth coastline)
freshwaterbody = gpd.read_file(r'ne_110m_rivers_lake_centerlines.shp')
gdf['proximity_to_freshwater_km'] = gdf.geometry.apply(
    lambda city: freshwaterbody.distance(city).min() * 111)  # Rough degree to km

**Distance to glacier (approx, using Natural Earth coastline)**

In [71]:
# Feature 3: Distance to freshwater (approx, using Natural Earth coastline)
glacier = gpd.read_file(r'ne_110m_glaciated_areas.shp')
gdf['proximity_to_glacier_km'] = gdf.geometry.apply(
    lambda city: glacier.distance(city).min() * 111)  # Rough degree to km

In [72]:
gdf = gdf.rename(columns={'name': 'city'})

In [87]:
gdf[df.isnull().any(axis=1)][:1]

,id,city,country,lat,lon,population,elevation,geometry,proximity_to_coast_km,proximity_to_freshwater_km,proximity_to_glacier_km,distance_temp
73,74,AmundsenScott South Pole Station,ATA,-89.98289,139.26699,200,NaN,POINT (139.26699 -89.98289),2488.967597,11624.024281,0.0,0.0


In [99]:

target_point = gdf[gdf.isnull().any(axis=1)].geometry.iloc[0]

# Calculate distances to all other points
gdf['distance_temp'] = gpd.GeoSeries(gdf.geometry, crs=4326).distance(target_point)

# Exclude the point itself and find the nearest
closest = gdf['distance_temp'][gdf['distance_temp'] > 0].sort_values().idxmin()

print(gdf.iloc[closest]['city'])
target_elevantion = gdf.iloc[closest]['elevation']
print(target_elevantion)
gdf.loc[df.isnull().any(axis=1), 'elevation'] = gdf.iloc[closest]['elevation']

Concordia Research Station
3231.0


<ipython-input-99-c4f3a61ba463>:4: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['distance_temp'] = gpd.GeoSeries(gdf.geometry, crs=4326).distance(target_point)


In [105]:
gdf[gdf.isnull().any(axis=1)][:1]
gdf.drop(columns=['distance_temp'], inplace=True)

In [106]:
print(gdf[['city', 'population', 'elevation', 'geometry', 'country', 'proximity_to_coast_km',
           'proximity_to_freshwater_km', 'proximity_to_glacier_km']])
gdf.to_csv('city_spatial.csv', index=True)

                city  population  elevation                     geometry  \
0              Bombo       75000     1191.0       POINT (32.5333 0.5833)   
1        Fort Portal       42670     1517.0         POINT (30.275 0.671)   
2            Potenza       69060      714.0        POINT (15.799 40.642)   
3         Campobasso       50762      768.0        POINT (14.656 41.563)   
4              Aosta       34062      575.0         POINT (7.315 45.737)   
...              ...         ...        ...                          ...   
1244  Rio de Janeiro    11748000       36.0  POINT (-43.22502 -22.92502)   
1245       Sao Paulo    18845000      729.0  POINT (-46.62502 -23.55868)   
1246          Sydney     4630000        7.0  POINT (151.18518 -33.92001)   
1247       Singapore     5183700       13.0    POINT (103.85582 1.29303)   
1248       Hong Kong     7206000       36.0   POINT (114.18501 22.30498)   

     country  proximity_to_coast_km  proximity_to_freshwater_km  \
0        UGA        

In [107]:
gdf_2 = gpd.read_file(r'city_spatial.csv')
gdf_2.head()

,field_1,id,city,country,lat,lon,population,elevation,geometry,proximity_to_coast_km,proximity_to_freshwater_km,proximity_to_glacier_km
0,0,1,Bombo,UGA,0.5833,32.5333,75000,1191.0,POINT (32.5333 0.5833),926.7780685420897,63.43062890564867,7669.969771931842
1,1,2,Fort Portal,UGA,0.671,30.275,42670,1517.0,POINT (30.275 0.671),1155.1143462899752,207.8861842073783,7688.633056299894
2,2,3,Potenza,ITA,40.642,15.799,69060,714.0,POINT (15.799 40.642),78.56226252073802,648.3363106982638,5421.587607284402
3,3,4,Campobasso,ITA,41.563,14.656,50762,768.0,POINT (14.656 41.563),66.93260942064187,658.3494009496502,5258.750499041271
4,4,5,Aosta,ITA,45.737,7.315,34062,575.0,POINT (7.315 45.737),207.88040818840935,275.35741813559787,4330.422574549758


In [ ]:
import folium

# Make an empty map
m = folium.Map(location=[4,47], tiles="OpenStreetMap", zoom_start=3, width='80%', height='600px')
for index, row in tqdm(gdf.iterrows()):
    folium.CircleMarker([row['geometry'].y, row['geometry'].x],
                        popup=row['city'],
                        radius=row['population'] / 1000000,
                        tooltip=f"{row['city']}<br>Population: {row['population']}").add_to(m)

m